In [1]:
import netsquid as ns
from netsquid.nodes import DirectConnection, Node
from netsquid.components import FibreDelayModel, FibreLossModel, QuantumChannel, Message
from netsquid.components import DepolarNoiseModel, DephaseNoiseModel
from netsquid.protocols import NodeProtocol


In [2]:
class SymmetricDirectConnection(DirectConnection):
    def __init__(self, name, L:int, loss_model, delay_model, noise_model) -> None:
        modelDict = {
                    'quantum_noise_model': noise_model,
                    'quantum_loss_model': loss_model,
                    'delay_model': delay_model
                    }
        abChannel = QuantumChannel("A->B", length=L, models=modelDict)
        baChannel = QuantumChannel("B->A", length=L, models=modelDict)
        super().__init__(name, abChannel, baChannel)

In [3]:
class SendProtocol(NodeProtocol):
    def __init__(self, node, stop_flag):
        super().__init__(node)
        self.qbitSent = 0
        self.stop_flag = stop_flag

    def run(self):
        port = self.node.ports["qubitIO"]
        while not self.stop_flag[0]:
            self.qbitSent += 1
            qubit_id = self.qbitSent

            # print(
            #     f"[{ns.sim_time(magnitude=ns.MICROSECOND)} μs] "
            #     f"{self.node.name} is sending qubit #{qubit_id}."
            # )

            qubit = ns.qubits.create_qubits(1)[0]
            msg = Message(items=[qubit], meta={"id": qubit_id})
            port.tx_output(msg)

            # yield self.await_port_output(self.node.ports["qubitIO"])
            yield self.await_timer(1e6)
            # yield self.await_signal()
            # Send (TX) qubit to the other node via port's output:
            # await_timer takes time in nanoseconds
            # yield self.await_timer(1e6)  # wait 1 microsecond before sending next qubit
            # print("B has received a qubit, stopping sender.")


class ReceiveProtocol(NodeProtocol):
    def __init__(self, node, stop_flag):
        super().__init__(node)
        self.arrival_time = None
        self.stop_flag = stop_flag
        self.received_id = None

    def run(self):
        port = self.node.ports["qubitIO"]
        # Wait (yield) until input has arrived on our port:
        yield self.await_port_input(port)
        self.stop_flag[0] = True
        current_time = ns.sim_time(magnitude=ns.MICROSECOND)
        self.arrival_time = current_time

        # We received a qubit.
        msg = port.rx_input()
        qubit = msg.items[0]  # Not needed
        self.received_id = msg.meta["meta"]["id"]

        # print received qubit id
        # print(f"Received qubit ID: {self.received_id}")

        # print(f"[{current_time} μs] {self.node.name} received a qubit.")
        # print(f"Received qubit: {received_qubit}")

In [4]:
def create_directConnected_nodes(distance: int, p: list[float], depolar_freq):
    assert len(p) >= 2
    portName = "qubitIO"
    nodeA = Node("nodeA", port_names=[portName])
    nodeB = Node("nodeB", port_names=[portName])
    conn = SymmetricDirectConnection("AB_channel", distance, 
                                     FibreLossModel(p[0], p[1]), FibreDelayModel(), 
                                     DepolarNoiseModel(depolar_freq))
    nodeA.connect_to(remote_node=nodeB, connection=conn,
                        local_port_name=portName, remote_port_name=portName)
    return nodeA, nodeB

In [5]:
results = []


def setup_sim():
    ns.sim_reset()

    node_distance = 100  # in km
    depolar_freq = 5e4  # depolarization frequency
    p_loss_init = 0.2  # initial loss probability
    p_loss_length = 0.2  # loss probability per km
    nodeA, nodeB = create_directConnected_nodes(
        node_distance, [p_loss_init, p_loss_length], depolar_freq
    )

    stop_flag = [False]  # Mutable flag to signal stopping
    AProtocol = SendProtocol(nodeA, stop_flag)
    BProtocol = ReceiveProtocol(nodeB, stop_flag)

    AProtocol.start()
    BProtocol.start()

    stats: ns.util.SimStats = ns.sim_run(magnitude=ns.MICROSECOND)
    # print(f"Simulation ended at t={ns.sim_time(magnitude=ns.MICROSECOND)} μs | {nodeA.name} sent {AProtocol.qbitSent} qubits")
    # print(stats.summary())
    # print("Raw data dictionary entries:")
    # print(stats.data)

    simulation_end_time = ns.sim_time(magnitude=ns.MICROSECOND)
    total_qubits_sent = BProtocol.received_id
    arrival_time = BProtocol.arrival_time

    results.append(
        (
            simulation_end_time,
            total_qubits_sent,
            arrival_time,
        )
    )


n_runs = 1

for _ in range(n_runs):
    setup_sim()


# results

In [ ]:
# median simulation end time, that still has the delay in it from the sender
simulation_end_times = [res[0] for res in results]
simulation_end_times.sort()
median_simulation_end_time = simulation_end_times[len(simulation_end_times) // 2]

print(f"Median simulation end time: {median_simulation_end_time} μs")

Median simulation end time: 38000.0 μs


In [7]:
# median qubits sent until reception
qubits_sent = [res[1] for res in results]
qubits_sent.sort()
median_qubits_sent = qubits_sent[len(qubits_sent) // 2]

print(f"Median qubits sent until reception: {median_qubits_sent}")

Median qubits sent until reception: 38


In [8]:
# median arrival time
arrival_times = [res[2] for res in results]
arrival_times.sort()
median_arrival_time = arrival_times[len(arrival_times) // 2]

print(f"Median arrival time: {median_arrival_time} μs")

Median arrival time: 37500.0 μs
